In [1]:
import ast
import pandas as pd
import glob

A function to read the json-like file spotify provides us

In [2]:
def load_file_to_df(file_path):
    records = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue  # skip empty lines
            try:
                # Convert the string representation to a dict
                record = ast.literal_eval(line)
                records.append(record)
            except Exception as e:
                print(f"Error processing line {i} in {file_path}: {e}")
    # Use json_normalize to flatten nested dictionaries
    df = pd.json_normalize(records)
    return df


# Spotify-Deezer matching data, but only the Spotify part

This is for the data pulled specificaly to pair Deezer and Spotify

In [4]:
df = load_file_to_df('../../Data/Raw/spotify_deezer_data_1.txt')
df.to_csv('../../Data/Raw/spotify_deezer_data_1.csv', index=False)

In [6]:
df = pd.read_csv('../../Data/Raw/spotify_deezer_data_1.csv')

# Data Cleaning

In [7]:
columns_to_keep = ['name', 'artists', 'album.name', 'duration_ms', 'popularity', 'id', 'external_ids.isrc', 'album.release_date_precision', 'album.release_date', 'album.total_tracks', 'disc_number', 'available_markets']
df_reduced = df[columns_to_keep]
df_reduced = df_reduced.drop_duplicates(subset='id', keep='first')
# Extract the first few artist names from the 'artists' column
df_reduced['artist_1'] = df_reduced['artists'].apply(lambda x: ast.literal_eval(x)[0]['name'] if len(ast.literal_eval(x)) > 0 else None)
df_reduced['artist_2'] = df_reduced['artists'].apply(lambda x: ast.literal_eval(x)[1]['name'] if len(ast.literal_eval(x)) > 1 else None)
df_reduced['artist_3'] = df_reduced['artists'].apply(lambda x: ast.literal_eval(x)[2]['name'] if len(ast.literal_eval(x)) > 2 else None)
df_reduced['artist_4'] = df_reduced['artists'].apply(lambda x: ast.literal_eval(x)[3]['name'] if len(ast.literal_eval(x)) > 3 else None)

# Drop the original 'artists' column (not for now, only once I'm fully sure how many artists we want to keep)
df_reduced = df_reduced.drop(columns=['artists'])

print(df_reduced[['artist_1', 'artist_2', 'artist_3', 'artist_4']].head())

                        artist_1 artist_2 artist_3 artist_4
0                          Márió     None     None     None
1  Count Basie And His Orhcestra     None     None     None
2                    Gene Ammons     None     None     None
3               The Funsong Band     None     None     None
4                Various Artists     None     None     None


In [8]:
df_reduced.to_csv('../../Data/Processed/spotify_deezer_data.csv', index=False)

# The Spotify Only Dataset

Outdated, was there when we were only pulling data from spotify, now also needs to filter the songs we took using isrc from deezer, so DO NOT RUN

In [ ]:
# I really should have shoved them into a single folder, but I can't be bothered to move them now - V
# Glob Glob Glob
all_files = glob.glob('../../Data/Raw/*.txt')  
dataframes = [load_file_to_df(file) for file in all_files if 'unique_songs' not in file]

# Combine all DataFrames into one
df_all = pd.concat(dataframes, ignore_index=True)

print(df_all.head())
df_all.describe(include='all')


In [7]:
df_all.to_csv('../../Data/Raw/spotify_data_merged.csv', index=False)

In [29]:
df_all = pd.read_csv('../../Data/Raw/spotify_data_merged.csv')

In [ ]:
# Find the maximum number of items in the 'artists' column
max_artists_count = df_reduced['artists'].apply(lambda x: len(ast.literal_eval(x))).max()
print(f"The maximum number of items in the 'artists' column is: {max_artists_count}")

10 artists for some songs? Do we want all of those?

In [ ]:
columns_to_keep = ['name', 'artists', 'album.name', 'duration_ms', 'popularity', 'id', 'external_ids.isrc', 'album.release_date_precision', 'album.release_date', 'album.total_tracks', 'disc_number', 'available_markets']
df_reduced = df_all[columns_to_keep]
df_reduced = df_reduced.drop_duplicates(subset='id', keep='first')

df_reduced['artist_1'] = df_reduced['artists'].apply(lambda x: ast.literal_eval(x)[0]['name'] if len(ast.literal_eval(x)) > 0 else None)
df_reduced['artist_2'] = df_reduced['artists'].apply(lambda x: ast.literal_eval(x)[1]['name'] if len(ast.literal_eval(x)) > 1 else None)
df_reduced['artist_3'] = df_reduced['artists'].apply(lambda x: ast.literal_eval(x)[2]['name'] if len(ast.literal_eval(x)) > 2 else None)
df_reduced['artist_4'] = df_reduced['artists'].apply(lambda x: ast.literal_eval(x)[3]['name'] if len(ast.literal_eval(x)) > 3 else None)


In [39]:
df_reduced.to_csv('../../Data/Processed/spotify_data.csv', index=False)